# M01 HW - Andrew Cornfeld (cpm6gh)

# Setup

In [1]:
import pandas as pd
import configparser

In [3]:
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

# Import File

In [4]:
!ls -l {data_home}

total 457
drwx--S--- 2 cpm6gh users   4096 Jan 16 16:05 gutenberg
-rw------- 1 cpm6gh users 465627 Jan 17 14:52 pg42324.txt


In [5]:
src_file = f"{data_home}/pg42324.txt"

In [6]:
lines = open(src_file, 'r').readlines()

In [7]:
lines[:5]

['\ufeffThe Project Gutenberg EBook of Frankenstein, by Mary W. Shelley\n',
 '\n',
 'This eBook is for the use of anyone anywhere at no cost and with\n',
 'almost no restrictions whatsoever.  You may copy it, give it away or\n',
 're-use it under the terms of the Project Gutenberg License included\n']

In [8]:
lines[-5:]

['\n',
 'This Web site includes information about Project Gutenberg-tm,\n',
 'including how to make donations to the Project Gutenberg Literary\n',
 'Archive Foundation, how to help produce our new eBooks, and how to\n',
 'subscribe to our email newsletter to hear about new eBooks.\n']

# Convert to Dataframe

In [9]:
text = pd.DataFrame(lines)

In [10]:
text

,0
0,"﻿The Project Gutenberg EBook of Frankenstein, ..."
1,\n
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...
...,...
8023,\n
8024,This Web site includes information about Proje...
8025,including how to make donations to the Project...
8026,"Archive Foundation, how to help produce our ne..."


In [13]:
text.sample(10)

,0
4233,"of De Lacey, when her attendant fell dangerous..."
2299,your return to us; and I was at first tempted ...
4048,was in height and proportion. I had never yet ...
3335,"only you and your family, but thousands of oth..."
4840,The thought was madness; it stirred the fiend ...
5698,"death-warrant, be sure that you are yourself s..."
698,\n
1477,distinguished discoverers. He then took a curs...
3046,ecstasy of our future prospects. The first of ...
5545,irksome to me. Sometimes I could not prevail o...


In [14]:
text.columns = ['line_str']

In [15]:
text.head()

,line_str
0,"﻿The Project Gutenberg EBook of Frankenstein, ..."
1,\n
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...


In [16]:
text.index.name = 'line_num'

In [17]:
text.head()

,line_str
line_num,
0,"﻿The Project Gutenberg EBook of Frankenstein, ..."
1,\n
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...


# 1. How many tokens does the raw text have?

In [18]:
text['len'] = text.line_str.str.len()

In [19]:
text.len.describe()

count    8028.000000
mean       56.996139
std        25.175999
min         1.000000
25%        63.000000
50%        69.000000
75%        72.000000
max        74.000000
Name: len, dtype: float64

## The raw text has 8028 tokens. 

In [20]:
chunk_pat = '\n\n'

In [21]:
chunks = open(src_file, 'r').read().split(chunk_pat)

In [22]:
chunks[:5]

['\ufeffThe Project Gutenberg EBook of Frankenstein, by Mary W. Shelley',
 'This eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org',
 '\nTitle: Frankenstein\n       or, The Modern Prometheus',
 'Author: Mary W. Shelley',
 'Release Date: March 13, 2013 [EBook #42324]']

In [24]:
text = pd.DataFrame(chunks, columns=['chunk_str'])
text.index.name = 'chunk_id'

In [25]:
text.head()

,chunk_str
chunk_id,
0,"﻿The Project Gutenberg EBook of Frankenstein, ..."
1,This eBook is for the use of anyone anywhere a...
2,"\nTitle: Frankenstein\n or, The Modern P..."
3,Author: Mary W. Shelley
4,"Release Date: March 13, 2013 [EBook #42324]"


In [26]:
text.shape

(942, 1)

In [27]:
text.chunk_str = text.chunk_str.str.replace('\n+', ' ', regex=True).str.strip()

In [28]:
text.head()

,chunk_str
chunk_id,
0,"﻿The Project Gutenberg EBook of Frankenstein, ..."
1,This eBook is for the use of anyone anywhere a...
2,"Title: Frankenstein or, The Modern Prom..."
3,Author: Mary W. Shelley
4,"Release Date: March 13, 2013 [EBook #42324]"


In [29]:
K = text.chunk_str.str.split(expand=True).stack().to_frame('token_str')
K.index.names = ['chunk_num', 'token_num']

In [30]:
K

token_str
chunk_num token_num           
0         0               ﻿The
          1            Project
          2          Gutenberg
          3              EBook
          4                 of
...                        ...
941       35                to
          36              hear
          37             about
          38               new
          39           eBooks.

[80985 rows x 1 columns]

In [31]:
K['term_str'] = K.token_str.str.replace(r'\W+', '', regex=True).str.lower()

In [32]:
K.sample(10)

,,token_str,term_str
chunk_num,token_num,,
482,142,massacring,massacring
519,9,not,not
471,50,her,her
289,26,the,the
816,87,during,during
99,17,and,and
208,269,I,i
617,98,resources,resources
744,53,heavenly,heavenly


# 2. What is the most frequent pronoun in the text?

In [33]:
V = K.term_str.value_counts().to_frame('n')
V.index.name = 'term_str'

In [35]:
V.head(15)

,n
term_str,
the,4575
and,3120
of,2918
i,2918
to,2257
my,1819
a,1497
in,1232
was,1064


## The most frequent pronoun in the text is 'me'.

# 3. Which subject pronoun is most frequent in the text we imported in class? 

## The most common subject pronoun in the text from class is 'she'.

# 4. Provide a brief explanation for this difference, based on what you may know about the two novels.

## I believe this is because Frankenstein is written in first person and Persuasion is written in third person point of view. The narrator in Frankenstein is talking about themselves, and therefore they would use the word 'me' frequently, whereas in Persuasion, the narrator is not a character in the story, so they would not mention themselves.